In [3]:
using Revise, LazySets, BenchmarkTools, SparseArrays

In [128]:
N = Float64
n = 100
cpa = CartesianProductArray([BallInf(zeros(N, 1), N(1)) for _ in 1:n])
dd = zeros(N, n); dd[1] = N(1); dd[3] = N(1)
ds = sparsevec([1, 3], N[1, 1], n)

100-element SparseVector{Float64,Int64} with 2 stored entries:
  [1  ]  =  1.0
  [3  ]  =  1.0

In [10]:
@btime ρ($dd, $cpa)
@btime ρ($ds, $cpa)  # master
@btime ρ_1729($ds, $cpa)  # schillic/1729

  2.632 μs (100 allocations: 9.38 KiB)
  5.312 μs (300 allocations: 18.81 KiB)
  162.706 ns (8 allocations: 640 bytes)


2.0

In [11]:
n = 4
cpa = CartesianProductArray([BallInf(zeros(N, 1), N(1)) for _ in 1:n])
dd = zeros(N, n); dd[1] = N(1); dd[3] = N(1)
ds = sparsevec([1, 3], N[1, 1], n);

4-element SparseVector{Float64,Int64} with 2 stored entries:
  [1]  =  1.0
  [3]  =  1.0

In [12]:
@btime ρ($dd, $cpa)
@btime ρ($ds, $cpa)  # master
@btime ρ_1729($ds, $cpa)  # schillic/1729

  110.427 ns (4 allocations: 384 bytes)
  227.469 ns (12 allocations: 832 bytes)
  161.528 ns (8 allocations: 640 bytes)


2.0

In [50]:
d = sprandn(10, 1, 0.2)[:]
cpa = CartesianProductArray([rand(Hyperrectangle, dim=2) for _ in 1:5]);

In [51]:
ρ(d, cpa)

-2.2455338704757626

In [52]:
ρ_1729(d, cpa)

-2.2455338704757626

In [53]:
@btime ρ($d, $cpa)

  291.853 ns (15 allocations: 1.00 KiB)


-2.2455338704757626

In [54]:
@btime ρ_1729($d, $cpa)

  169.510 ns (8 allocations: 672 bytes)


-2.2455338704757626

In [55]:
d

10-element SparseVector{Float64,Int64} with 3 stored entries:
  [2 ]  =  0.995627
  [3 ]  =  -0.932751
  [4 ]  =  1.49306

In [57]:
indices, _ = findnz(d)

([2, 3, 4], [0.9956265124250439, -0.9327508849583663, 1.4930602425207065])

In [58]:
next_idx = 1
next_dim = indices[next_idx]

2

In [59]:
m = length(indices)

3

In [62]:
i0 = 1 # index of starting block

1

In [63]:
i1 = i0 + dim(cpa.array[1]) - 1 # index of final index

2

## support function

In [7]:
# faster version for sparse vectors
function ρ_1729(d::AbstractSparseVector{N}, cpa::CartesianProductArray{N}) where {N<:Real}
    sfun = zero(N)
    indices, _ = SparseArrays.findnz(d)
    if isempty(indices)
        # direction is the zero vector
        return sfun
    end
    next_idx = 1
    next_dim = indices[next_idx]
    m = length(indices)
    i0 = 1
    for Xi in cpa.array
        i1 = i0 + dim(Xi) - 1
        if next_dim <= i1
            # there is a non-zero entry in this block
            sfun += ρ(d[i0:i1], Xi)

            # find next index outside the current block
            next_idx += 1
            while next_idx <= m && indices[next_idx] <= i1
                next_idx += 1
            end
            if next_idx > m
                # no more non-zero entries
                break
            end
            next_dim = indices[next_idx]
        end
        i0 = i1 + 1
    end
    return sfun
end

ρ_1729 (generic function with 1 method)

In [ ]:
function ρ_master(d::AbstractVector{N}, cpa::CartesianProductArray{N}) where {N<:Real}
    sfun = zero(N)
    i0 = 1
    for Xi in cpa.array
        i1 = i0 + dim(Xi) - 1
        sfun += ρ(d[i0:i1], Xi)
        i0 = i1 + 1
    end
    return sfun
end

## support vector

In [8]:
# faster version for sparse vectors
function σ_1729(d::AbstractSparseVector{N}, cpa::CartesianProductArray{N}
          ) where {N<:Real}
    svec = similar(d)
    indices, _ = SparseArrays.findnz(d)
    if isempty(indices)
        # direction is the zero vector
        return an_element(cpa)
    end
    next_idx = 1
    next_dim = indices[next_idx]
    m = length(indices)
    i0 = 1
    for Xi in cpa.array
        i1 = i0 + dim(Xi) - 1
        if next_dim <= i1
            # there is a non-zero entry in this block
            svec[i0:i1] = σ(d[i0:i1], Xi)

            # find next index outside the current block
            next_idx += 1
            while next_idx <= m && indices[next_idx] <= i1
                next_idx += 1
            end
            if next_idx <= m
                next_dim = indices[next_idx]
            end
        else
            svec[i0:i1] = an_element(Xi)
        end
        i0 = i1 + 1
    end
    return svec
end

σ_1729 (generic function with 1 method)

## single entries

In [129]:
using LazySets: SingleEntryVector

dse = SingleEntryVector(100, 100, 2.0)
dsp = sparsevec([100], [2.0], 100)
cpa = CartesianProductArray([rand(Hyperrectangle, dim=2) for _ in 1:50]);

In [130]:
dse.i

100

In [131]:
function ρ_single_entry(d::SingleEntryVector{N}, cpa::CartesianProductArray{N}) where {N}
    i0 = 1
    idx = d.i
    for Xi in cpa.array
        i1 = i0 + dim(Xi) - 1
        if i0 <= idx && idx <= i1
            return ρ(@view(d[i0:i1]), Xi)
        end
        i0 = i1 + 1
    end
    return sfun  
endusing LazySets: SingleEntryVector

dse = SingleEntryVector(100, 100, 2.0)
dsp = sparsevec([100], [2.0], 100)
cpa = CartesianProductArray([rand(Hyperrectangle, dim=2) for _ in 1:50]);

ρ_single_entry (generic function with 1 method)

In [138]:
using LazySets: SingleEntryVector

dse = SingleEntryVector(1, 100, 2.0)
dsp = sparsevec([1], [2.0], 100)
cpa = CartesianProductArray([rand(Hyperrectangle, dim=2) for _ in 1:50]);

In [139]:
@btime ρ_single_entry($dse, $cpa)
@btime ρ_1729($dsp, $cpa)

  13.776 ns (1 allocation: 16 bytes)
  113.257 ns (5 allocations: 416 bytes)


-0.5162752710697882